In [ ]:
# 경고 메시지가 나오지 않도록 warnings 모듈을 사용합니다.
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

In [ ]:
# 아나콘다의 경우 아래와 같이 코딩합니다.
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

# 컴퓨터에 있는 맑은 고딕 폰트를 사용합니다.
f_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

In [ ]:
# cp949 방식으로 읽어야 합니다.
df = pd.read_csv('17.data.csv', encoding='cp949')
df.head()

In [ ]:
# 컬럼 이름을 확인합니다.
df.columns

In [ ]:
# 필요한 컬럼만 선택합니다.
df = df[['사업장명','사업장업종코드','사업장업종코드명','법정동주소광역시도코드','가입자수','신규취득자수','상실가입자수','당월고지금액']].copy()
df 

In [ ]:
# 컬럼 이름을 사용하기 쉽게 바꿉니다.
column_names = ['사업장명','업종코드','업종코드명','시도코드','가입자수','신규','상실','금액']
df.columns = column_names
df.head()

In [ ]:
# 사업자 이름에서 필요한 것을 지우기 위해서 정규식을 사용합니다.
import re

pattern1 = '(\([^)]+\))'
re.sub(pattern1, '', '파이썬회사(주)')

In [ ]:
# 정규식 패턴을 정합니다.
pattern1 = '(\([^)]+\))'
pattern2 = '(\[[^)]+\])'
pattern3 = '[^A-Za-z0-9가-힣]'

In [ ]:
# 함수를 사용해서 필요없는 글자를 삭제합니다.
def preprocessing(x):
    x = re.sub(pattern1, '', x)
    x = re.sub(pattern2, '', x)
    x = re.sub(pattern3, ' ', x)
    x = re.sub(' +', ' ', x)
    return x

df['사업장명'] = df['사업장명'].apply(preprocessing)
df.head()

In [ ]:
# 수도권 기업 가입자수를 확인해보겠습니다.
# 내림차순으로 20개를 보여줍니다.
# 시도코드가 11~41 사이에 있으면 수도권에 있는 기업입니다.
plt.figure(figsize=(8, 10))
tmp = df.loc[df['시도코드'].isin([11, 41]), ['사업장명', '가입자수']].sort_values('가입자수', ascending=False).head(20).reset_index()
sns.barplot(y='사업장명', x='가입자수', data=tmp)
plt.xticks(rotation=30)
plt.title('수도권 기업 가입자수')
plt.show()

* 국민연금 보험률은 9%입니다. 
    * 급여의 9%를 국민연금으로 냅니다.
* 4.5%는 회사가, 나머지 4.5%는 개인이 부담합니다.
* 이를 거꾸로 계산해서 급여를 계산합니다.
* 국민연금 보험료는 소득 상한선과 하한선이 설정되어 있습니다.
* 하지만 상한선과 하한선이 있어서 실제보다 작게 계산될 수 있습니다.

> * 1인당금액 = 금액 / 가입자수   
> * 월급여 = 1인당금액 / 9% * 100%   
> * 연봉  = 월급여 * 12개월

In [ ]:
df['1인당금액'] = df['금액'] / df['가입자수']
df['월급여'] =  df['1인당금액'] / 9 * 100
df['연봉'] = df['월급여'] * 12
df.head()

In [ ]:
# 결측치를 제거합니다.
df.dropna(axis=0, inplace=True)
df

In [ ]:
# 회사 정보를 알려주는 함수를 만듭니다.
def find_company(name):
    return df.loc[df['사업장명']==name, ['사업장명','가입자수','연봉']].sort_values('연봉', ascending=False)

In [ ]:
find_company('삼성전자')

In [ ]:
# 파일로 저장합니다.
df.to_csv('17.data_processing.csv')